In [1]:
# Import necessary libraries for accessing and parsing web pages

from bs4 import BeautifulSoup
import requests
import re

In [2]:
# Specify the base url and browser for accessing websites

HEADERS = {"User-Agent": "Mozilla/5.0"}
BASE_URL = "https://en.wikipedia.org"
HIGHEST_GROSSING_FILMS_URL = "/wiki/List_of_highest-grossing_films"

In [3]:
# Access page and check if it was successful

page = requests.get(BASE_URL + HIGHEST_GROSSING_FILMS_URL, headers=HEADERS)
assert page.status_code == 200, "Failed to access page"

In [4]:
# Parse the page content using BeautifulSoup
soup = BeautifulSoup(page.content, "lxml")

In [5]:
# Extract the table containing the highest grossing films
highest_grossing_films_table = soup.find("table", {"class": "wikitable sortable plainrowheaders sticky-header col4right col5center col6center"})

In [6]:
# Obtain film titles from the table, skipping the first row which contains column names 
film_titles = [highest_grossing_films_table.select("tr")[i].select("th")[0].a.text for i in range(1, len(highest_grossing_films_table.select("tr")))]

In [7]:
# Obtain release year for each film from the table
film_release_years = [highest_grossing_films_table.select("tr")[i].select("td")[3].text.strip() for i in range(1, len(highest_grossing_films_table.select("tr")))]

In [8]:
# We will need link for each film to p=obtain some specific information for each film
film_links = [BASE_URL + highest_grossing_films_table.select("tr")[i].select("th")[0].a.get("href") for i in range(1, len(highest_grossing_films_table.select("tr")))]

In [9]:
# Create list of directors and extract the director(s) for each film
directors = []

for link in film_links:
    # Access film page and search for directors
    film_page = requests.get(link, headers=HEADERS)
    film_soup = BeautifulSoup(film_page.content, "lxml")
    # Extract target element
    raw_string_director = film_soup.find("th", string="Directed by").find_next_sibling("td").text
    # Clean up the string and split it into a list
    
    # Delete references and special symbols and add commas between names
    current_directors = re.sub(r'([a-z])([A-Z])',  r'\1,\2', re.sub(r'\[\d+\]', ",", raw_string_director.replace("\n", "")))
    
    # If the last character is a comma, remove it
    if current_directors[-1] == ",":
        current_directors = current_directors[:-1]
        
    # Split the string into a list of directors
    current_directors = current_directors.split(",")
    
    directors.append(current_directors)

In [10]:
# Obtain revenues for each film from the table 
films_box_office_revenues = [re.sub(r'^.*\$', '$', highest_grossing_films_table.select("tr")[i].select("td")[2].text.strip()) for i in range(1, len(highest_grossing_films_table.select("tr")))]

In [ ]:
# Find country for each film accessing its web page
film_countries = []

for link in film_links:
    film_page = requests.get(link, headers=HEADERS)
    film_soup = BeautifulSoup(film_page.content, "lxml")
    # Find "country" or "countries" element and access its neighbor
    country_td = film_soup.find("th", string=re.compile(r"Country|Countries")).find_next_sibling("td")
    
    # If we see Unordered List, we will extract countries from it
    if country_td.find("ul"):
    # Extract every country from the list
        current_countries = [li.get_text() for li in country_td.find_all("li")]
    
    # IF there is no list, we will extract countries directly
    else:
        current_countries = country_td.get_text(",", strip=True).split(",")
    
    # Clean string from references and special symbols and separate when seeing a capital letter after a lowercase letter
    current_countries = [re.sub(r'([a-z])([A-Z])',  r'\1,\2', re.sub(r'\[\d+\]', '', current_country.replace("\n", ""))) for current_country in current_countries]
    
    film_countries.append(current_countries)

In [12]:
# Import libraries for data manipulation and storage
import sqlite3
import json

In [13]:
# Connect to a database and create a cursor for interacting with it
connection = sqlite3.connect("films.db")
cursor = connection.cursor()

In [14]:
# Query to create a table for storing film data
create_films_table = """CREATE TABLE IF NOT EXISTS 
films(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    release_year INTEGER,
    director TEXT,
    box_office_revenue TEXT,
    country TEXT
)
"""

# Execute the query
cursor.execute(create_films_table)

In [15]:
# Insert data into the table
for title, release_year, directors, revenue, country in zip(film_titles, film_release_years, directors, films_box_office_revenues, film_countries):
    cursor.execute("INSERT INTO films (title, release_year, director, box_office_revenue, country) VALUES (?, ?, ?, ?, ?)", (title, release_year, ", ".join(directors), revenue, ", ".join(country)))

In [16]:
# Extract data from the table
cursor.execute("SELECT * FROM films")

# Fetch data from the cursor
films_data = cursor.fetchall()

In [17]:
# Obtain column names
column_names = [description[0] for description in cursor.description]

# Transform films_data into a list of dictionaries (JSON format)
films_list = [dict(zip(column_names, film)) for film in films_data]

# Write film data to a JSON file
with open("films.json", "w") as file:
    json.dump(films_list, file, indent=4)